# Iris classification
https://github.com/jwheeldon/test_ml.git

Machine learning test using the following techniques on iris dataset:
* Descriptive statistics
* Linear support vector machine (SVM)
* Evaluation of model

In [43]:
# Import packages
from sklearn import datasets, svm, metrics
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import matplotlib as mpl

## Import and sample dataset

In [44]:
iris = datasets.load_iris()
n_samples = len(iris.data)
data_and_classes = list(zip(iris.data, iris.target))
data_pd = pd.DataFrame(data_and_classes)
print(data_pd.head(5))

                      0  1
0  [5.1, 3.5, 1.4, 0.2]  0
1  [4.9, 3.0, 1.4, 0.2]  0
2  [4.7, 3.2, 1.3, 0.2]  0
3  [4.6, 3.1, 1.5, 0.2]  0
4  [5.0, 3.6, 1.4, 0.2]  0


Reshape data and assign column to each feature.

In [45]:
data_pd = pd.concat([data_pd[0].apply(pd.Series), data_pd[1]], axis=1)
data_pd.columns = [1,2,3,4,5]

## Explore data using descriptive statistics

The first four columns [0:5] represent Iris Sepal Length, Sepal Width, Petal Length and Petal Width. The last column is represents iris species Setosa (0), Versicolour (1), and Virginica (2).

In [46]:
data_pd.describe()

,1,2,3,4,5
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,1.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


Stratify dataset by species - check for sampling bias, skewness

In [76]:
data_pd[[1,2]].groupby(data_pd[5]).describe()

1                                                 2                   \
  count   mean       std  min    25%  50%  75%  max count   mean       std   
5                                                                            
0  50.0  5.006  0.352490  4.3  4.800  5.0  5.2  5.8  50.0  3.418  0.381024   
1  50.0  5.936  0.516171  4.9  5.600  5.9  6.3  7.0  50.0  2.770  0.313798   
2  50.0  6.588  0.635880  4.9  6.225  6.5  6.9  7.9  50.0  2.974  0.322497   

                                
   min    25%  50%    75%  max  
5                               
0  2.3  3.125  3.4  3.675  4.4  
1  2.0  2.525  2.8  3.000  3.4  
2  2.2  2.800  3.0  3.175  3.8

In [77]:
data_pd[[3,4]].groupby(data_pd[5]).describe()

3                                                  4                   \
  count   mean       std  min  25%   50%    75%  max count   mean       std   
5                                                                             
0  50.0  1.464  0.173511  1.0  1.4  1.50  1.575  1.9  50.0  0.244  0.107210   
1  50.0  4.260  0.469911  3.0  4.0  4.35  4.600  5.1  50.0  1.326  0.197753   
2  50.0  5.552  0.551895  4.5  5.1  5.55  5.875  6.9  50.0  2.026  0.274650   

                            
   min  25%  50%  75%  max  
5                           
0  0.1  0.2  0.2  0.3  0.6  
1  1.0  1.2  1.3  1.5  1.8  
2  1.4  1.8  2.0  2.3  2.5

## Shuffle dataset
Since iris data is ordered by species, shuffling rows will help split training vs test data.

In [26]:
np.random.seed(10)
data_pd = data_pd.sample(frac=1)


print(data_pd.head(5))

       1    2    3    4  5
87   6.3  2.3  4.4  1.3  1
111  6.4  2.7  5.3  1.9  2
10   5.4  3.7  1.5  0.2  0
91   6.1  3.0  4.6  1.4  1
49   5.0  3.3  1.4  0.2  0


## Define training data and targets
Split data into two groups: 50% model training - 50% test data. Best practice would typically involve a split of 70% training - 10% validation - 20% test data.

In [27]:
training = data_pd[:][:n_samples//2]
training_data = training[[1,2,3,4]].values
training_target = training[5].values

print(training_data[0:10])
print(training_target[0:10])

[[ 6.3  2.3  4.4  1.3]
 [ 6.4  2.7  5.3  1.9]
 [ 5.4  3.7  1.5  0.2]
 [ 6.1  3.   4.6  1.4]
 [ 5.   3.3  1.4  0.2]
 [ 5.   2.   3.5  1. ]
 [ 6.3  2.5  4.9  1.5]
 [ 5.8  2.7  4.1  1. ]
 [ 5.1  3.4  1.5  0.2]
 [ 5.7  2.8  4.5  1.3]]
[1 2 0 1 0 1 1 1 0 1]


## Support vector machine classification

Use an unsupervised linear SVM to predict the species of unlabelled data.

In [28]:
# Linear support vector machine classifier
clf = svm.LinearSVC()

# Train model to fit training_data to training_target
clf.fit(training_data, training_target)

# Predict iris.targets and define expected vs predicted classifications
expected = data_pd[5][n_samples//2:].values
predicted = clf.predict(data_pd[[1,2,3,4]][n_samples//2:])

In [29]:
# Generate confusion matrix and classification report via metrics
print(metrics.confusion_matrix(expected,predicted))

[[27  0  0]
 [ 0 23  0]
 [ 0  3 22]]


Accuracy score: Determines the accuracy of classification by comparing two arrays for an exact match.

Classification report: Precision (P) reflects true positive events, while Recall (R) represents true negative events.
F1-score is defined as the harmonic mean between P and R. Support relfects the number of true cases per class.



![image](http://scikit-learn.org/stable/_images/math/4427313bbf584ad5dc95320f3fe1f8356b3c5f9f.png)
![image](http://scikit-learn.org/stable/_images/math/3bcc92cd7cbd26a3d6c6aa0b407cb9d7c89c6254.png)
![image](http://scikit-learn.org/stable/_images/math/6d03e9528e504c0801b231b90bdb480e01269ea1.png)

In [30]:
print("Accuracy score:", metrics.accuracy_score(expected, predicted))
print(metrics.classification_report(expected, predicted))

Accuracy score: 0.96
             precision    recall  f1-score   support

          0       1.00      1.00      1.00        27
          1       0.88      1.00      0.94        23
          2       1.00      0.88      0.94        25

avg / total       0.96      0.96      0.96        75

